In [18]:
import tensorflow as tf
import pandas as pd

CSV_COLUMN_NAMES = ['x', 'y', 'z', 'rotX', 'rotY','rotZ','solution']
SPECIES = ['dusmedi', 'dustu']

train = pd.read_csv("Blocks/Train.txt", names=CSV_COLUMN_NAMES, header=0)
test = pd.read_csv("Blocks/Test.txt", names=CSV_COLUMN_NAMES, header=0)

train_y = train.pop('solution')
test_y = test.pop('solution')
train.head()

,x,y,z,rotX,rotY,rotZ
0,0.923845,2.196076,1,0,-1,0
1,0.433130,0.996076,1,0,-37,0
2,0.923845,2.196076,1,0,-1,0
3,0.433130,0.988228,1,0,-37,0
4,0.923845,2.188228,1,0,-1,0


In [21]:

def input_evaluation_set():
    features = {'x': np.array([0.923845,0.433130]),
                'y':  np.array([2.196076,0.996076]),
                'z': np.array([1, 1]),
                'rotX':  np.array([0, 0]),
                'rotY':  np.array([-1,-37]),
                'rotZ':  np.array([0,0])}
    labels = np.array([2, 1])
    return features, labels

def input_fn(features, labels, training=True, batch_size=256):
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))
    if training:
        dataset = dataset.shuffle(1000).repeat()
    
    return dataset.batch(batch_size)

my_feature_columns = []
for key in train.keys():
    my_feature_columns.append(tf.feature_column.numeric_column(key=key))
    
       
    
classifier = tf.estimator.DNNClassifier(
    feature_columns=my_feature_columns,
    hidden_units=[30, 10],
    n_classes=2)


# Train the Model.
classifier.train(
    input_fn=lambda: input_fn(train, train_y, training=True),
    steps=5000)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\admin\\AppData\\Local\\Temp\\tmp58nofdhj', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000013B96396748>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
Instructions for 

INFO:tensorflow:loss = 0.25539663, step = 4700 (0.380 sec)
INFO:tensorflow:global_step/sec: 260.435
INFO:tensorflow:loss = 0.25047496, step = 4800 (0.384 sec)
INFO:tensorflow:global_step/sec: 261.114
INFO:tensorflow:loss = 0.25068378, step = 4900 (0.382 sec)
INFO:tensorflow:Saving checkpoints for 5000 into C:\Users\admin\AppData\Local\Temp\tmp58nofdhj\model.ckpt.
INFO:tensorflow:Loss for final step: 0.2504742.


In [22]:
eval_result = classifier.evaluate(
    input_fn=lambda: input_fn(test, test_y, training=False))

print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))

INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-12-05T17:44:46Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\admin\AppData\Local\Temp\tmp58nofdhj\model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-12-05-17:44:48
INFO:tensorflow:Saving dict for global step 5000: accuracy = 1.0, accuracy_baseline = 1.0, a

In [27]:
# Generate predictions from the model
expected = ['dusmedi', 'dusmedi','dustu']
predict_x = {
    'x': [0.9238452,0.4331303,1.9238452],
    'y': [2.117596,0.890128,2.058736],
    'z': [1, 1, 1],
    'rotX': [0, 0, 0],
    'rotY': [-1,-37,-1],
    'rotZ': [0,0,0]
}

def input_fn(features, batch_size=256):
    return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)

predictions = classifier.predict(
    input_fn=lambda: input_fn(predict_x))



for pred_dict, expec in zip(predictions, expected):
    class_id = pred_dict['class_ids'][0]
    probability = pred_dict['probabilities'][class_id]

    print('Prediction is "{}" ({:.1f}%), expected "{}"'.format(
        SPECIES[class_id], 100 * probability, expec))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\admin\AppData\Local\Temp\tmp58nofdhj\model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Prediction is "dusmedi" (65.5%), expected "dusmedi"
Prediction is "dusmedi" (97.3%), expected "dusmedi"
Prediction is "dusmedi" (54.0%), expected "dustu"
